Given the results of the Survey in a CSV

- Check it and clean it
- Creates survey ratings df
- Integrate it with the ml ratings df
- Export the resulting csv

form based on https://docs.google.com/forms/d/1M1qkwX0DpBi7SI2b13Iuk10iQ0nBH_HvCBcEGCUjNBg/edit#responses

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)

### preparing survey data

In [2]:
# import Survey results CSV
path = "../data/"
file = "Movie Matching 2010-2019.csv"

survey = pd.read_csv(path+file)

In [3]:
# drop first 2 rows: they were testers
# set names as id and place them in the index
survey = (survey.drop([0,1])
                .rename(columns = {"Please, add a name or alias":"userId"})
                .reset_index(drop=True)
                .set_index("userId"))

In [4]:
# store in a series additional commentaries in the dataset
additional = survey["Additional films that you would like to remark"].dropna()

# drop unnecessary columns
drop_columns = ["Timestamp", "Unnamed: 101", "Unnamed: 102", "Unnamed: 103", "Additional films that you would like to remark"]

survey = survey.drop(columns=drop_columns)

In [5]:
# Create table with stats
movies_stats = survey.describe().T

users_stats = survey.T.describe().T

In [6]:
# create df with NaN values as 0
survey_matrix = survey.fillna(0)

### Melting Survey dataframe

In [7]:
# melt(inverse of pivot) the survey file to have the survey_ratings file
# define a list with every title name
melt_columns = list(survey.columns)

# create intermediate column for adding userId as column
# (problems melting with the index value)
survey_x = survey.copy()
survey_x["userId"]=survey_x.index

# creat melt
survey_melt = pd.melt(survey_x, id_vars="userId", value_vars=melt_columns)

In [8]:
# subset only movies with review
# rename columns name

survey_melt = (survey_melt[-survey_melt["value"].isna()]
                                                .reset_index(drop=True)
                                                .rename(columns={"variable":"title_survey", "value":"rating"}))

In [9]:
survey_melt

,userId,title_survey,rating
0,Mano,Joker (2019),5.0
1,olivia,Joker (2019),5.0
2,Seno,Joker (2019),4.0
3,Sujit,Joker (2019),4.0
4,brecht,Joker (2019),5.0
...,...,...,...
1725,Wim Wenders,The Social Network (2010),5.0
1726,Isabel,The Social Network (2010),2.0
1727,Julie,The Social Network (2010),3.0
1728,Tony Stark,The Social Network (2010),3.0


### import movie characteristics and creating survey_ratings df

In [10]:
# import the movies data from the survey
file2 = "list of movies - dataset.csv"

survey_movie_list = pd.read_csv(path+file2)

In [11]:
# merge survey_melt with survey_movie_list
# subset only the some columns

survey_ratings = survey_melt.merge(survey_movie_list, how="inner", left_on = "title_survey", right_on = "title_survey")[["userId", "imdbId", "rating"]]

In [12]:
survey_ratings

,userId,imdbId,rating
0,Mano,7286456,5.0
1,olivia,7286456,5.0
2,Seno,7286456,4.0
3,Sujit,7286456,4.0
4,brecht,7286456,5.0
...,...,...,...
1671,Wim Wenders,1285016,5.0
1672,Isabel,1285016,2.0
1673,Julie,1285016,3.0
1674,Tony Stark,1285016,3.0


In [13]:
# there are movies from 2019 that are not included in the movie lens dataset

file4 = "list of movies - missing_movies_from_ml_imdbIds.csv"

missing_movies = pd.read_csv(path+file4)

In [14]:
# create a subset of the survey_ratings with every movie till 2018 (included in the ml dataset)
survey_ratings_2018 = survey_ratings[-survey_ratings["imdbId"].isin(missing_movies["imdbId"])].reset_index(drop=True)

### subsetting movieLens ratings only with movies of the survey

not used on the final project, problems to upload 700MB file into the github.

Only used locally

#### import the movies data from the survey
path2 = "../data/ml-latest/"
file3 = "links.csv"
file4 = "ratings.csv"

ml_links = pd.read_csv(path2+file3)
ml_ratings = pd.read_csv(path2+file4)

#### subset ml_ratings only with imdbId
ml_ratings = ml_ratings.merge(ml_links, how="inner", left_on="movieId", right_on="movieId")[["userId", "imdbId", "rating"]]

#### define a list of movies
list_of_movies = list(survey_movie_list["imdbId"])
ml_ratings_sv = ml_ratings[ml_ratings["imdbId"].isin(list_of_movies)]

#### creating boolean variable to both ratings
ml_ratings_sv["source"]=0
survey_ratings["source"]=1

#### concat both ratings in one dataset
ratings_merged = pd.concat([survey_ratings, ml_ratings_sv],sort=False).reset_index(drop=True)

### export csv of all value tables

movies_stats.to_csv(path+"movies_stats.csv")

users_stats.to_csv(path+"users_stats.csv")

###### survey_matrix.to_csv(path+"survey_matrix.csv", index=False)
survey_matrix.to_csv(path+"survey_matrix.csv")

survey_ratings.to_csv(path+"survey_ratings.csv", index=False)

ratings_merged.to_csv(path+"ratings_merged.csv", index=False)

#### concat both with only with the movies till 2018
ratings_merged_2018 = pd.concat([survey_ratings_2018, ml_ratings_sv],sort=False).reset_index(drop=True)

ratings_merged_2018.to_csv(path+"ratings_merged_2018.csv", index=False)